# Calculating Hotel Occupancy

---

> Hotel occupancy is a critical factor during the booking process and can provide additional insight into the likelihood of cancellations and/or forecasting future ADR.
> 
> However, *there's no clear indication of the total number of guest rooms for either hotel.*
>
> 
> I will determine the maximum number of rooms occupied for each date for each hotel, which can be used as a placeholder max occupancy number.

---

# Import Packages and Read Data

In [86]:
## Data Handling
import datetime as dt
import pandas as pd
import numpy as np
import sweetviz as sv

## Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_rows', 50)
%matplotlib inline

## Load Pre-Reviewed Data

In [87]:
data_path = '../../data/5.2_exploded_temporal_update.parquet'
df_data = pd.read_parquet(data_path)
df_data

,IsCanceled,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,UUID,ArrivalDate,LoS_Numeric,LoS_Days,DepartureDate,Date,BookingDate,DaysSinceBooking,ArrivalDate_DaysBeforeHoliday,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,Date_DaysBeforeHoliday,Date_DaysAfterHoliday,DateWeek,DateDay,DateDayName,DateMonthName
0,0,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,9af79666-f290-45c5-868c-2f9601b8f98b,2015-07-01,0,0 days,2015-07-01,2015-07-01,2014-07-24,342,45,21,45,21,22,44,45,21,27,3,Wednesday,July
1,0,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,81440274-e84e-4502-89f3-e01681d0672a,2015-07-01,0,0 days,2015-07-01,2015-07-01,2013-06-24,737,45,21,45,21,52,14,45,21,27,3,Wednesday,July
2,0,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NULL,NULL,0,Transient,75.00,0,0,Check-Out,H1,60fe936c-f7ba-48d9-ac73-71c21e1b3978,2015-07-01,1,1 days,2015-07-02,2015-07-01,2015-06-24,8,45,21,44,22,52,14,45,21,27,3,Wednesday,July
3,0,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,NULL,0,Transient,75.00,0,0,Check-Out,H1,5b2aae61-1d0c-4314-b4c1-603595e43163,2015-07-01,1,1 days,2015-07-02,2015-07-01,2015-06-18,14,45,21,44,22,58,8,45,21,27,3,Wednesday,July
4,0,2015,July,27,1,0,1,2,0.00,0,BB,IRL,Online TA,TA/TO,0,0,0,A,E,0,No Deposit,240,NULL,0,Transient,88.20,0,0,Check-Out,H1,8225801b-2e43-490a-9703-4fec1baa34d8,2015-07-01,1,1 days,2015-07-02,2015-07-01,2015-06-19,13,45,21,44,22,57,9,45,21,27,3,Wednesday,July
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528642,0,2017,August,35,31,4,10,2,0.00,0,HB,DEU,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,69,NULL,0,Transient,99.06,0,0,Check-Out,H1,8ebd4e66-52a2-4630-a636-b3da3297918d,2017-08-31,14,14 days,2017-09-14,2017-09-12,2017-03-23,175,35,16,21,30,22,81,23,28,37,2,Tuesday,September
528643,0,2017,August,35,31,4,10,2,0.00,0,HB,GBR,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,40,NULL,0,Contract,112.80,0,1,Check-Out,H1,5370a7d8-c7ba-41d2-92b5-3ce1c3469c5a,2017-08-31,14,14 days,2017-09-14,2017-09-13,2017-02-01,225,35,16,21,30,72,31,22,29,37,3,Wednesday,September
528644,0,2017,August,35,31,4,10,2,0.00,0,HB,DEU,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,69,NULL,0,Transient,99.06,0,0,Check-Out,H1,8ebd4e66-52a2-4630-a636-b3da3297918d,2017-08-31,14,14 days,2017-09-14,2017-09-13,2017-03-23,175,35,16,21,30,22,81,22,29,37,3,Wednesday,September
528645,0,2017,August,35,31,4,10,2,0.00,0,HB,GBR,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,40,NULL,0,Contract,112.80,0,1,Check-Out,H1,5370a7d8-c7ba-41d2-92b5-3ce1c3469c5a,2017-08-31,14,14 days,2017-09-14,2017-09-14,2017-02-01,225,35,16,21,30,72,31,21,30,37,4,Thursday,September


In [88]:
df_data = df_data.set_index('Date')
df_data

,IsCanceled,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,UUID,ArrivalDate,LoS_Numeric,LoS_Days,DepartureDate,BookingDate,DaysSinceBooking,ArrivalDate_DaysBeforeHoliday,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,Date_DaysBeforeHoliday,Date_DaysAfterHoliday,DateWeek,DateDay,DateDayName,DateMonthName
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-07-01,0,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,9af79666-f290-45c5-868c-2f9601b8f98b,2015-07-01,0,0 days,2015-07-01,2014-07-24,342,45,21,45,21,22,44,45,21,27,3,Wednesday,July
2015-07-01,0,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,81440274-e84e-4502-89f3-e01681d0672a,2015-07-01,0,0 days,2015-07-01,2013-06-24,737,45,21,45,21,52,14,45,21,27,3,Wednesday,July
2015-07-01,0,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NULL,NULL,0,Transient,75.00,0,0,Check-Out,H1,60fe936c-f7ba-48d9-ac73-71c21e1b3978,2015-07-01,1,1 days,2015-07-02,2015-06-24,8,45,21,44,22,52,14,45,21,27,3,Wednesday,July
2015-07-01,0,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,NULL,0,Transient,75.00,0,0,Check-Out,H1,5b2aae61-1d0c-4314-b4c1-603595e43163,2015-07-01,1,1 days,2015-07-02,2015-06-18,14,45,21,44,22,58,8,45,21,27,3,Wednesday,July
2015-07-01,0,2015,July,27,1,0,1,2,0.00,0,BB,IRL,Online TA,TA/TO,0,0,0,A,E,0,No Deposit,240,NULL,0,Transient,88.20,0,0,Check-Out,H1,8225801b-2e43-490a-9703-4fec1baa34d8,2015-07-01,1,1 days,2015-07-02,2015-06-19,13,45,21,44,22,57,9,45,21,27,3,Wednesday,July
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-12,0,2017,August,35,31,4,10,2,0.00,0,HB,DEU,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,69,NULL,0,Transient,99.06,0,0,Check-Out,H1,8ebd4e66-52a2-4630-a636-b3da3297918d,2017-08-31,14,14 days,2017-09-14,2017-03-23,175,35,16,21,30,22,81,23,28,37,2,Tuesday,September
2017-09-13,0,2017,August,35,31,4,10,2,0.00,0,HB,GBR,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,40,NULL,0,Contract,112.80,0,1,Check-Out,H1,5370a7d8-c7ba-41d2-92b5-3ce1c3469c5a,2017-08-31,14,14 days,2017-09-14,2017-02-01,225,35,16,21,30,72,31,22,29,37,3,Wednesday,September
2017-09-13,0,2017,August,35,31,4,10,2,0.00,0,HB,DEU,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,69,NULL,0,Transient,99.06,0,0,Check-Out,H1,8ebd4e66-52a2-4630-a636-b3da3297918d,2017-08-31,14,14 days,2017-09-14,2017-03-23,175,35,16,21,30,22,81,22,29,37,3,Wednesday,September


# Calculate Daily Occupancies

## Filter for `Check-Out` Reservations

In [89]:
# non_cxl_res = df_data['ReservationStatus'] == 'Check-Out'
# df_data = df_data[non_cxl_res]
# df_data

In [90]:
df_data['IsCanceled'].value_counts()

IsCanceled
0    330206
1    198441
Name: count, dtype: int64

# Hotel 1

In [91]:
df_data['HotelNumber'].unique()

array(['H1', 'H2'], dtype=object)

In [92]:
hotel = 'H1'

hotel_filter = (df_data['HotelNumber'] == hotel)

df_data_h1 = df_data[hotel_filter]

df_data_h1

,IsCanceled,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,UUID,ArrivalDate,LoS_Numeric,LoS_Days,DepartureDate,BookingDate,DaysSinceBooking,ArrivalDate_DaysBeforeHoliday,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,Date_DaysBeforeHoliday,Date_DaysAfterHoliday,DateWeek,DateDay,DateDayName,DateMonthName
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-07-01,0,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,9af79666-f290-45c5-868c-2f9601b8f98b,2015-07-01,0,0 days,2015-07-01,2014-07-24,342,45,21,45,21,22,44,45,21,27,3,Wednesday,July
2015-07-01,0,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,81440274-e84e-4502-89f3-e01681d0672a,2015-07-01,0,0 days,2015-07-01,2013-06-24,737,45,21,45,21,52,14,45,21,27,3,Wednesday,July
2015-07-01,0,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NULL,NULL,0,Transient,75.00,0,0,Check-Out,H1,60fe936c-f7ba-48d9-ac73-71c21e1b3978,2015-07-01,1,1 days,2015-07-02,2015-06-24,8,45,21,44,22,52,14,45,21,27,3,Wednesday,July
2015-07-01,0,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,NULL,0,Transient,75.00,0,0,Check-Out,H1,5b2aae61-1d0c-4314-b4c1-603595e43163,2015-07-01,1,1 days,2015-07-02,2015-06-18,14,45,21,44,22,58,8,45,21,27,3,Wednesday,July
2015-07-01,0,2015,July,27,1,0,1,2,0.00,0,BB,IRL,Online TA,TA/TO,0,0,0,A,E,0,No Deposit,240,NULL,0,Transient,88.20,0,0,Check-Out,H1,8225801b-2e43-490a-9703-4fec1baa34d8,2015-07-01,1,1 days,2015-07-02,2015-06-19,13,45,21,44,22,57,9,45,21,27,3,Wednesday,July
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-12,0,2017,August,35,31,4,10,2,0.00,0,HB,DEU,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,69,NULL,0,Transient,99.06,0,0,Check-Out,H1,8ebd4e66-52a2-4630-a636-b3da3297918d,2017-08-31,14,14 days,2017-09-14,2017-03-23,175,35,16,21,30,22,81,23,28,37,2,Tuesday,September
2017-09-13,0,2017,August,35,31,4,10,2,0.00,0,HB,GBR,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,40,NULL,0,Contract,112.80,0,1,Check-Out,H1,5370a7d8-c7ba-41d2-92b5-3ce1c3469c5a,2017-08-31,14,14 days,2017-09-14,2017-02-01,225,35,16,21,30,72,31,22,29,37,3,Wednesday,September
2017-09-13,0,2017,August,35,31,4,10,2,0.00,0,HB,DEU,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,69,NULL,0,Transient,99.06,0,0,Check-Out,H1,8ebd4e66-52a2-4630-a636-b3da3297918d,2017-08-31,14,14 days,2017-09-14,2017-03-23,175,35,16,21,30,22,81,22,29,37,3,Wednesday,September


In [93]:
## Filter for non-canceled reservations

# Step 1: Filter the DataFrame to include only "Check-Out" reservations
check_out_reservations = df_data_h1[df_data_h1['ReservationStatus'] == 'Check-Out']

# Step 2: Calculate the total number of "Check-Out" reservations for each date
total_check_out_reservations_per_date = check_out_reservations.groupby('Date').size().reset_index(name='TotalCheckOutReservations')

# Step 3: Determine the maximum possible "Check-Out" reservations for any given date
max_possible_check_out_reservations = total_check_out_reservations_per_date['TotalCheckOutReservations'].max()

# Step 4: Calculate the percentage occupancy for each date
total_check_out_reservations_per_date['PercentageOccupancy'] = (total_check_out_reservations_per_date['TotalCheckOutReservations'] / max_possible_check_out_reservations) * 100

total_check_out_reservations_per_date = total_check_out_reservations_per_date.set_index('Date')
total_check_out_reservations_per_date

,TotalCheckOutReservations,PercentageOccupancy
Date,,
2015-07-01,38,13.57
2015-07-02,71,25.36
2015-07-03,91,32.50
2015-07-04,117,41.79
2015-07-05,145,51.79
...,...,...
2017-09-10,7,2.50
2017-09-11,3,1.07
2017-09-12,3,1.07


In [94]:
df_data_h1 = pd.concat([df_data_h1, total_check_out_reservations_per_date], axis=1)
df_data_h1

,IsCanceled,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,UUID,ArrivalDate,LoS_Numeric,LoS_Days,DepartureDate,BookingDate,DaysSinceBooking,ArrivalDate_DaysBeforeHoliday,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,Date_DaysBeforeHoliday,Date_DaysAfterHoliday,DateWeek,DateDay,DateDayName,DateMonthName,TotalCheckOutReservations,PercentageOccupancy
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-07-01,0,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,9af79666-f290-45c5-868c-2f9601b8f98b,2015-07-01,0,0 days,2015-07-01,2014-07-24,342,45,21,45,21,22,44,45,21,27,3,Wednesday,July,38,13.57
2015-07-01,0,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,81440274-e84e-4502-89f3-e01681d0672a,2015-07-01,0,0 days,2015-07-01,2013-06-24,737,45,21,45,21,52,14,45,21,27,3,Wednesday,July,38,13.57
2015-07-01,0,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NULL,NULL,0,Transient,75.00,0,0,Check-Out,H1,60fe936c-f7ba-48d9-ac73-71c21e1b3978,2015-07-01,1,1 days,2015-07-02,2015-06-24,8,45,21,44,22,52,14,45,21,27,3,Wednesday,July,38,13.57
2015-07-01,0,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,NULL,0,Transient,75.00,0,0,Check-Out,H1,5b2aae61-1d0c-4314-b4c1-603595e43163,2015-07-01,1,1 days,2015-07-02,2015-06-18,14,45,21,44,22,58,8,45,21,27,3,Wednesday,July,38,13.57
2015-07-01,0,2015,July,27,1,0,1,2,0.00,0,BB,IRL,Online TA,TA/TO,0,0,0,A,E,0,No Deposit,240,NULL,0,Transient,88.20,0,0,Check-Out,H1,8225801b-2e43-490a-9703-4fec1baa34d8,2015-07-01,1,1 days,2015-07-02,2015-06-19,13,45,21,44,22,57,9,45,21,27,3,Wednesday,July,38,13.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-12,0,2017,August,35,31,4,10,2,0.00,0,HB,DEU,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,69,NULL,0,Transient,99.06,0,0,Check-Out,H1,8ebd4e66-52a2-4630-a636-b3da3297918d,2017-08-31,14,14 days,2017-09-14,2017-03-23,175,35,16,21,30,22,81,23,28,37,2,Tuesday,September,3,1.07
2017-09-13,0,2017,August,35,31,4,10,2,0.00,0,HB,GBR,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,40,NULL,0,Contract,112.80,0,1,Check-Out,H1,5370a7d8-c7ba-41d2-92b5-3ce1c3469c5a,2017-08-31,14,14 days,2017-09-14,2017-02-01,225,35,16,21,30,72,31,22,29,37,3,Wednesday,September,2,0.71
2017-09-13,0,2017,August,35,31,4,10,2,0.00,0,HB,DEU,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,69,NULL,0,Transient,99.06,0,0,Check-Out,H1,8ebd4e66-52a2-4630-a636-b3da3297918d,2017-08-31,14,14 days,2017-09-14,2017-03-23,175,35,16,21,30,22,81,22,29,37,3,Wednesday,September,2,0.71


In [95]:
df_data_h1

,IsCanceled,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,UUID,ArrivalDate,LoS_Numeric,LoS_Days,DepartureDate,BookingDate,DaysSinceBooking,ArrivalDate_DaysBeforeHoliday,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,Date_DaysBeforeHoliday,Date_DaysAfterHoliday,DateWeek,DateDay,DateDayName,DateMonthName,TotalCheckOutReservations,PercentageOccupancy
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-07-01,0,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,9af79666-f290-45c5-868c-2f9601b8f98b,2015-07-01,0,0 days,2015-07-01,2014-07-24,342,45,21,45,21,22,44,45,21,27,3,Wednesday,July,38,13.57
2015-07-01,0,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,81440274-e84e-4502-89f3-e01681d0672a,2015-07-01,0,0 days,2015-07-01,2013-06-24,737,45,21,45,21,52,14,45,21,27,3,Wednesday,July,38,13.57
2015-07-01,0,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NULL,NULL,0,Transient,75.00,0,0,Check-Out,H1,60fe936c-f7ba-48d9-ac73-71c21e1b3978,2015-07-01,1,1 days,2015-07-02,2015-06-24,8,45,21,44,22,52,14,45,21,27,3,Wednesday,July,38,13.57
2015-07-01,0,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,NULL,0,Transient,75.00,0,0,Check-Out,H1,5b2aae61-1d0c-4314-b4c1-603595e43163,2015-07-01,1,1 days,2015-07-02,2015-06-18,14,45,21,44,22,58,8,45,21,27,3,Wednesday,July,38,13.57
2015-07-01,0,2015,July,27,1,0,1,2,0.00,0,BB,IRL,Online TA,TA/TO,0,0,0,A,E,0,No Deposit,240,NULL,0,Transient,88.20,0,0,Check-Out,H1,8225801b-2e43-490a-9703-4fec1baa34d8,2015-07-01,1,1 days,2015-07-02,2015-06-19,13,45,21,44,22,57,9,45,21,27,3,Wednesday,July,38,13.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-12,0,2017,August,35,31,4,10,2,0.00,0,HB,DEU,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,69,NULL,0,Transient,99.06,0,0,Check-Out,H1,8ebd4e66-52a2-4630-a636-b3da3297918d,2017-08-31,14,14 days,2017-09-14,2017-03-23,175,35,16,21,30,22,81,23,28,37,2,Tuesday,September,3,1.07
2017-09-13,0,2017,August,35,31,4,10,2,0.00,0,HB,GBR,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,40,NULL,0,Contract,112.80,0,1,Check-Out,H1,5370a7d8-c7ba-41d2-92b5-3ce1c3469c5a,2017-08-31,14,14 days,2017-09-14,2017-02-01,225,35,16,21,30,72,31,22,29,37,3,Wednesday,September,2,0.71
2017-09-13,0,2017,August,35,31,4,10,2,0.00,0,HB,DEU,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,69,NULL,0,Transient,99.06,0,0,Check-Out,H1,8ebd4e66-52a2-4630-a636-b3da3297918d,2017-08-31,14,14 days,2017-09-14,2017-03-23,175,35,16,21,30,22,81,22,29,37,3,Wednesday,September,2,0.71


# Hotel 2

In [96]:
hotel = 'H2'

hotel_filter = (df_data['HotelNumber'] == hotel)

df_data_h2 = df_data[hotel_filter]

df_data_h2

,IsCanceled,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,UUID,ArrivalDate,LoS_Numeric,LoS_Days,DepartureDate,BookingDate,DaysSinceBooking,ArrivalDate_DaysBeforeHoliday,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,Date_DaysBeforeHoliday,Date_DaysAfterHoliday,DateWeek,DateDay,DateDayName,DateMonthName
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-07-01,0,2015,July,27,1,0,2,1,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,0.00,0,0,Check-Out,H2,100de9b6-d997-4b66-93bf-1bef01047f01,2015-07-01,2,2 days,2015-07-03,2015-06-25,8,45,21,43,23,51,15,45,21,27,3,Wednesday,July
2015-07-01,1,2015,July,27,1,0,2,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,1,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Canceled,H2,214a6ead-c43b-4c62-92e2-3bed955ea609,2015-07-01,2,2 days,2015-07-03,2014-10-17,242,45,21,43,23,52,63,45,21,27,3,Wednesday,July
2015-07-01,1,2015,July,27,1,0,2,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,1,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Canceled,H2,811c60e0-8904-4dfb-b092-3930186827b2,2015-07-01,2,2 days,2015-07-03,2014-10-17,242,45,21,43,23,52,63,45,21,27,3,Wednesday,July
2015-07-01,1,2015,July,27,1,0,2,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,1,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Canceled,H2,75472c42-a9d0-4f8e-8e3c-f19f50d8cc56,2015-07-01,2,2 days,2015-07-03,2014-10-17,242,45,21,43,23,52,63,45,21,27,3,Wednesday,July
2015-07-01,1,2015,July,27,1,0,2,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,1,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Canceled,H2,29d40459-75c8-478a-a8c0-f8e52326a23a,2015-07-01,2,2 days,2015-07-03,2014-10-17,242,45,21,43,23,52,63,45,21,27,3,Wednesday,July
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-08,1,2017,August,35,28,4,10,3,0.00,0,BB,MOZ,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9,NULL,0,Transient,200.29,0,1,Canceled,H2,55864a77-de45-4b11-a3b6-5aebe20361ed,2017-08-28,14,14 days,2017-09-11,2017-06-19,3,38,13,24,27,57,4,27,24,36,5,Friday,September
2017-09-09,1,2017,August,35,28,3,9,1,0.00,0,BB,BEL,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9,NULL,0,Transient,161.83,0,0,Canceled,H2,acfdbed1-fd7a-4493-a1f6-f145f51f0871,2017-08-28,12,12 days,2017-09-09,2017-07-18,6,38,13,26,25,28,33,26,25,36,6,Saturday,September
2017-09-09,1,2017,August,35,28,4,10,3,0.00,0,BB,MOZ,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9,NULL,0,Transient,200.29,0,1,Canceled,H2,55864a77-de45-4b11-a3b6-5aebe20361ed,2017-08-28,14,14 days,2017-09-11,2017-06-19,3,38,13,24,27,57,4,26,25,36,6,Saturday,September


In [97]:
## Filter for non-canceled reservations

# Step 1: Filter the DataFrame to include only "Check-Out" reservations
check_out_reservations = df_data_h2[df_data_h2['ReservationStatus'] == 'Check-Out']

# Step 2: Calculate the total number of "Check-Out" reservations for each date
total_check_out_reservations_per_date = check_out_reservations.groupby('Date').size().reset_index(name='TotalCheckOutReservations')

# Step 3: Determine the maximum possible "Check-Out" reservations for any given date
max_possible_check_out_reservations = total_check_out_reservations_per_date['TotalCheckOutReservations'].max()

# Step 4: Calculate the percentage occupancy for each date
total_check_out_reservations_per_date['PercentageOccupancy'] = (total_check_out_reservations_per_date['TotalCheckOutReservations'] / max_possible_check_out_reservations) * 100

total_check_out_reservations_per_date = total_check_out_reservations_per_date.set_index('Date')
total_check_out_reservations_per_date

,TotalCheckOutReservations,PercentageOccupancy
Date,,
2015-07-01,65,18.57
2015-07-02,66,18.86
2015-07-03,76,21.71
2015-07-04,22,6.29
2015-07-05,22,6.29
...,...,...
2017-09-03,85,24.29
2017-09-04,35,10.00
2017-09-05,9,2.57


In [98]:
df_data_h2 = pd.concat([df_data_h2, total_check_out_reservations_per_date], axis=1)
df_data_h2

,IsCanceled,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,UUID,ArrivalDate,LoS_Numeric,LoS_Days,DepartureDate,BookingDate,DaysSinceBooking,ArrivalDate_DaysBeforeHoliday,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,Date_DaysBeforeHoliday,Date_DaysAfterHoliday,DateWeek,DateDay,DateDayName,DateMonthName,TotalCheckOutReservations,PercentageOccupancy
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-07-01,0,2015,July,27,1,0,2,1,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,0.00,0,0,Check-Out,H2,100de9b6-d997-4b66-93bf-1bef01047f01,2015-07-01,2,2 days,2015-07-03,2015-06-25,8,45,21,43,23,51,15,45,21,27,3,Wednesday,July,65.00,18.57
2015-07-01,1,2015,July,27,1,0,2,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,1,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Canceled,H2,214a6ead-c43b-4c62-92e2-3bed955ea609,2015-07-01,2,2 days,2015-07-03,2014-10-17,242,45,21,43,23,52,63,45,21,27,3,Wednesday,July,65.00,18.57
2015-07-01,1,2015,July,27,1,0,2,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,1,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Canceled,H2,811c60e0-8904-4dfb-b092-3930186827b2,2015-07-01,2,2 days,2015-07-03,2014-10-17,242,45,21,43,23,52,63,45,21,27,3,Wednesday,July,65.00,18.57
2015-07-01,1,2015,July,27,1,0,2,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,1,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Canceled,H2,75472c42-a9d0-4f8e-8e3c-f19f50d8cc56,2015-07-01,2,2 days,2015-07-03,2014-10-17,242,45,21,43,23,52,63,45,21,27,3,Wednesday,July,65.00,18.57
2015-07-01,1,2015,July,27,1,0,2,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,1,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Canceled,H2,29d40459-75c8-478a-a8c0-f8e52326a23a,2015-07-01,2,2 days,2015-07-03,2014-10-17,242,45,21,43,23,52,63,45,21,27,3,Wednesday,July,65.00,18.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-08,1,2017,August,35,28,4,10,3,0.00,0,BB,MOZ,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9,NULL,0,Transient,200.29,0,1,Canceled,H2,55864a77-de45-4b11-a3b6-5aebe20361ed,2017-08-28,14,14 days,2017-09-11,2017-06-19,3,38,13,24,27,57,4,27,24,36,5,Friday,September,NaN,NaN
2017-09-09,1,2017,August,35,28,3,9,1,0.00,0,BB,BEL,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9,NULL,0,Transient,161.83,0,0,Canceled,H2,acfdbed1-fd7a-4493-a1f6-f145f51f0871,2017-08-28,12,12 days,2017-09-09,2017-07-18,6,38,13,26,25,28,33,26,25,36,6,Saturday,September,NaN,NaN
2017-09-09,1,2017,August,35,28,4,10,3,0.00,0,BB,MOZ,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9,NULL,0,Transient,200.29,0,1,Canceled,H2,55864a77-de45-4b11-a3b6-5aebe20361ed,2017-08-28,14,14 days,2017-09-11,2017-06-19,3,38,13,24,27,57,4,26,25,36,6,Saturday,September,NaN,NaN


In [99]:
df_data_h2

,IsCanceled,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,UUID,ArrivalDate,LoS_Numeric,LoS_Days,DepartureDate,BookingDate,DaysSinceBooking,ArrivalDate_DaysBeforeHoliday,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,Date_DaysBeforeHoliday,Date_DaysAfterHoliday,DateWeek,DateDay,DateDayName,DateMonthName,TotalCheckOutReservations,PercentageOccupancy
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-07-01,0,2015,July,27,1,0,2,1,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,0.00,0,0,Check-Out,H2,100de9b6-d997-4b66-93bf-1bef01047f01,2015-07-01,2,2 days,2015-07-03,2015-06-25,8,45,21,43,23,51,15,45,21,27,3,Wednesday,July,65.00,18.57
2015-07-01,1,2015,July,27,1,0,2,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,1,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Canceled,H2,214a6ead-c43b-4c62-92e2-3bed955ea609,2015-07-01,2,2 days,2015-07-03,2014-10-17,242,45,21,43,23,52,63,45,21,27,3,Wednesday,July,65.00,18.57
2015-07-01,1,2015,July,27,1,0,2,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,1,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Canceled,H2,811c60e0-8904-4dfb-b092-3930186827b2,2015-07-01,2,2 days,2015-07-03,2014-10-17,242,45,21,43,23,52,63,45,21,27,3,Wednesday,July,65.00,18.57
2015-07-01,1,2015,July,27,1,0,2,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,1,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Canceled,H2,75472c42-a9d0-4f8e-8e3c-f19f50d8cc56,2015-07-01,2,2 days,2015-07-03,2014-10-17,242,45,21,43,23,52,63,45,21,27,3,Wednesday,July,65.00,18.57
2015-07-01,1,2015,July,27,1,0,2,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,1,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Canceled,H2,29d40459-75c8-478a-a8c0-f8e52326a23a,2015-07-01,2,2 days,2015-07-03,2014-10-17,242,45,21,43,23,52,63,45,21,27,3,Wednesday,July,65.00,18.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-08,1,2017,August,35,28,4,10,3,0.00,0,BB,MOZ,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9,NULL,0,Transient,200.29,0,1,Canceled,H2,55864a77-de45-4b11-a3b6-5aebe20361ed,2017-08-28,14,14 days,2017-09-11,2017-06-19,3,38,13,24,27,57,4,27,24,36,5,Friday,September,NaN,NaN
2017-09-09,1,2017,August,35,28,3,9,1,0.00,0,BB,BEL,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9,NULL,0,Transient,161.83,0,0,Canceled,H2,acfdbed1-fd7a-4493-a1f6-f145f51f0871,2017-08-28,12,12 days,2017-09-09,2017-07-18,6,38,13,26,25,28,33,26,25,36,6,Saturday,September,NaN,NaN
2017-09-09,1,2017,August,35,28,4,10,3,0.00,0,BB,MOZ,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9,NULL,0,Transient,200.29,0,1,Canceled,H2,55864a77-de45-4b11-a3b6-5aebe20361ed,2017-08-28,14,14 days,2017-09-11,2017-06-19,3,38,13,24,27,57,4,26,25,36,6,Saturday,September,NaN,NaN


In [100]:
full_dataset = pd.concat([df_data_h1, df_data_h2], axis = 0)
full_dataset

,IsCanceled,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,UUID,ArrivalDate,LoS_Numeric,LoS_Days,DepartureDate,BookingDate,DaysSinceBooking,ArrivalDate_DaysBeforeHoliday,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,Date_DaysBeforeHoliday,Date_DaysAfterHoliday,DateWeek,DateDay,DateDayName,DateMonthName,TotalCheckOutReservations,PercentageOccupancy
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-07-01,0,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,9af79666-f290-45c5-868c-2f9601b8f98b,2015-07-01,0,0 days,2015-07-01,2014-07-24,342,45,21,45,21,22,44,45,21,27,3,Wednesday,July,38.00,13.57
2015-07-01,0,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,81440274-e84e-4502-89f3-e01681d0672a,2015-07-01,0,0 days,2015-07-01,2013-06-24,737,45,21,45,21,52,14,45,21,27,3,Wednesday,July,38.00,13.57
2015-07-01,0,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NULL,NULL,0,Transient,75.00,0,0,Check-Out,H1,60fe936c-f7ba-48d9-ac73-71c21e1b3978,2015-07-01,1,1 days,2015-07-02,2015-06-24,8,45,21,44,22,52,14,45,21,27,3,Wednesday,July,38.00,13.57
2015-07-01,0,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,NULL,0,Transient,75.00,0,0,Check-Out,H1,5b2aae61-1d0c-4314-b4c1-603595e43163,2015-07-01,1,1 days,2015-07-02,2015-06-18,14,45,21,44,22,58,8,45,21,27,3,Wednesday,July,38.00,13.57
2015-07-01,0,2015,July,27,1,0,1,2,0.00,0,BB,IRL,Online TA,TA/TO,0,0,0,A,E,0,No Deposit,240,NULL,0,Transient,88.20,0,0,Check-Out,H1,8225801b-2e43-490a-9703-4fec1baa34d8,2015-07-01,1,1 days,2015-07-02,2015-06-19,13,45,21,44,22,57,9,45,21,27,3,Wednesday,July,38.00,13.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-08,1,2017,August,35,28,4,10,3,0.00,0,BB,MOZ,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9,NULL,0,Transient,200.29,0,1,Canceled,H2,55864a77-de45-4b11-a3b6-5aebe20361ed,2017-08-28,14,14 days,2017-09-11,2017-06-19,3,38,13,24,27,57,4,27,24,36,5,Friday,September,NaN,NaN
2017-09-09,1,2017,August,35,28,3,9,1,0.00,0,BB,BEL,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9,NULL,0,Transient,161.83,0,0,Canceled,H2,acfdbed1-fd7a-4493-a1f6-f145f51f0871,2017-08-28,12,12 days,2017-09-09,2017-07-18,6,38,13,26,25,28,33,26,25,36,6,Saturday,September,NaN,NaN
2017-09-09,1,2017,August,35,28,4,10,3,0.00,0,BB,MOZ,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9,NULL,0,Transient,200.29,0,1,Canceled,H2,55864a77-de45-4b11-a3b6-5aebe20361ed,2017-08-28,14,14 days,2017-09-11,2017-06-19,3,38,13,24,27,57,4,26,25,36,6,Saturday,September,NaN,NaN


In [101]:
full_dataset.to_parquet('../../data/5.3_exploded_data_with_occupancies.parquet', compression = 'zstd')